# Идея проекта, выборка и предобработка данных
<hr>

# Обработка данных
<hr>

> Данные сос

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [629]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [630]:
import pandas as pd
import numpy as np

In [631]:
df = pd.read_json("../dfdc_dataframes/df_total_0_4_compact.json")

In [632]:
# Балансировка классов 1:1

df = pd.concat([
    df_[df_.fake==True].sample(
        int(df_[df_.fake==False].shape[0]*1)), 
    df_[df_.fake==False]
])
print(df.shape)
df.head()

(990, 479)


,index,filename,fake,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_8_hist_noise_3,pt_9_hist_noise_3,pt_10_hist_noise_3,pt_11_hist_noise_3,pt_12_hist_noise_3,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3
868,72,wjqqxxabcv.mp4,True,14.155245,23.416007,30.739396,32.142585,28.426580,21.294063,16.556163,...,2.888889,6.888889,9.444444,10.666667,9.00000,10.000000,10.777778,10.555556,10.777778,0.143157
818,22,vbhsyoigsu.mp4,True,20.952636,17.619485,29.935506,36.756329,40.638065,33.864972,16.511481,...,5.029586,2.071006,3.106509,13.757396,16.12426,12.278107,15.384615,13.165680,15.532544,0.149303
2899,280,xvwzcguypx.mp4,True,20.790623,14.928868,4.811688,2.912795,13.554145,21.277556,14.709160,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,4.345703
3194,575,aaaoqepxnf.mp4,True,19.067431,18.991576,20.713449,26.899396,27.074729,31.968046,27.526805,...,6.250000,8.000000,3.500000,4.500000,22.00000,16.750000,14.500000,13.500000,16.500000,0.312438
2273,265,exbcottrza.mp4,True,14.219705,15.198947,13.453349,11.558026,14.447135,19.461382,11.929532,...,12.244898,10.204082,8.673469,6.632653,2.55102,5.102041,9.183673,32.653061,24.489796,0.659180


In [633]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,overall_face_std,overall_face_var,overall_face_hist_std,overall_face_hist_var,overall_face_hist_noise_0.5,overall_face_hist_noise_1,overall_face_hist_noise_3
fake,,,,,,,
True,45.378596,2245.551116,543.314786,420324.209,0.174416,0.251753,0.287317


In [634]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,overall_face_std,overall_face_var,overall_face_hist_std,overall_face_hist_var,overall_face_hist_noise_0.5,overall_face_hist_noise_1,overall_face_hist_noise_3
fake,,,,,,,
False,49.182392,2608.860611,534.771606,433193.597027,0.200743,0.275005,0.308146


In [635]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_37_var,886.436591,599.238627,0.193311,287.197964
pt_43_var,655.061498,447.013656,0.188778,208.047842
pt_38_var,703.751271,482.816710,0.186196,220.934560
pt_52_var,672.672529,461.699856,0.185982,210.972673
pt_58_hist_var,37.547698,54.627500,0.185297,17.079802
pt_36_var,792.309635,549.145733,0.181269,243.163902
pt_37_hist_var,21.588196,31.108482,0.180662,9.520285
pt_0_var,1394.724571,979.087061,0.175093,415.637509
pt_57_hist_var,41.762928,58.728494,0.168826,16.965566
pt_50_var,799.494447,570.495301,0.167154,228.999146


In [636]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
overall_face_hist_noise_0.5,0.200743,0.174416,0.375159
overall_face_hist_noise_1,0.275005,0.251753,0.526758
overall_face_hist_noise_3,0.308146,0.287317,0.595464
pt_7_hist_noise_0.5,1.449724,1.571798,3.021522
pt_8_hist_noise_0.5,1.452484,1.577242,3.029726
pt_9_hist_noise_0.5,1.483363,1.583386,3.066750
pt_10_hist_noise_0.5,1.474690,1.799517,3.274208
pt_6_hist_noise_0.5,1.742977,1.984977,3.727954
pt_24_hist_noise_0.5,2.536658,2.381511,4.918169
pt_56_hist_noise_0.5,2.480267,2.573087,5.053355


In [637]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_37_var,287.197964,1485.675218,0.193311
pt_43_var,208.047842,1102.075154,0.188778
pt_38_var,220.934560,1186.567981,0.186196
pt_52_var,210.972673,1134.372384,0.185982
pt_58_hist_var,17.079802,92.175198,0.185297
pt_36_var,243.163902,1341.455368,0.181269
pt_37_hist_var,9.520285,52.696678,0.180662
pt_0_var,415.637509,2373.811632,0.175093
pt_57_hist_var,16.965566,100.491423,0.168826
pt_50_var,228.999146,1369.989748,0.167154


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [638]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(5).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(5).index) +
    list(df_smart.filter(like='std', axis=0).head(3).index) 
))

features

['overall_face_hist_var',
 'overall_face_hist_noise_3',
 'overall_face_hist_noise_1',
 'pt_6_hist_noise_0.5',
 'pt_7_hist_noise_3',
 'pt_8_hist_noise_3',
 'pt_58_hist_std',
 'pt_21_hist_noise_0.5',
 'overall_face_hist_std',
 'pt_38_hist_noise_3',
 'pt_10_hist_noise_0.5',
 'pt_25_hist_noise_0.5',
 'pt_0_std',
 'pt_37_std',
 'pt_43_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'pt_66_hist_noise_3']

## Разбивка данных на train и test сплиты

In [639]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,pt_55_std,...,pt_8_hist_noise_3,pt_9_hist_noise_3,pt_10_hist_noise_3,pt_11_hist_noise_3,pt_12_hist_noise_3,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3
868,72,True,14.155245,23.416007,30.739396,32.142585,28.426580,21.294063,16.556163,14.388877,...,2.888889,6.888889,9.444444,10.666667,9.00000,10.000000,10.777778,10.555556,10.777778,0.143157
818,22,True,20.952636,17.619485,29.935506,36.756329,40.638065,33.864972,16.511481,30.751520,...,5.029586,2.071006,3.106509,13.757396,16.12426,12.278107,15.384615,13.165680,15.532544,0.149303
2899,280,True,20.790623,14.928868,4.811688,2.912795,13.554145,21.277556,14.709160,22.100534,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,4.345703
3194,575,True,19.067431,18.991576,20.713449,26.899396,27.074729,31.968046,27.526805,26.218844,...,6.250000,8.000000,3.500000,4.500000,22.00000,16.750000,14.500000,13.500000,16.500000,0.312438
2273,265,True,14.219705,15.198947,13.453349,11.558026,14.447135,19.461382,11.929532,12.891567,...,12.244898,10.204082,8.673469,6.632653,2.55102,5.102041,9.183673,32.653061,24.489796,0.659180


In [640]:
from sklearn.model_selection import train_test_split

In [641]:
X_train, X_test, y_train, y_test = train_test_split( 
    df_compact[features], 
    df_compact['fake'],
    test_size=0.20, 
    random_state=42)

In [642]:
X_train.head()

,overall_face_hist_var,overall_face_hist_noise_3,overall_face_hist_noise_1,pt_6_hist_noise_0.5,pt_7_hist_noise_3,pt_8_hist_noise_3,pt_58_hist_std,pt_21_hist_noise_0.5,overall_face_hist_std,pt_38_hist_noise_3,pt_10_hist_noise_0.5,pt_25_hist_noise_0.5,pt_0_std,pt_37_std,pt_43_hist_noise_3,overall_face_hist_noise_0.5,pt_66_hist_noise_3
2266,1.325267e+05,0.224941,0.210998,3.645833,19.444444,12.152778,3.496650,2.430556,364.042161,15.625000,0.347222,0.868056,45.983039,34.412580,8.333333,0.153369,15.451389
3074,1.314330e+04,0.541950,0.541950,0.000000,25.000000,31.632653,1.366361,0.000000,114.644252,40.816327,0.000000,0.000000,55.908144,36.445529,36.224490,0.285714,43.877551
2700,4.609439e+05,0.159544,0.131144,0.520833,3.993056,3.298611,5.171285,3.125000,678.928496,8.680556,0.347222,1.562500,23.803278,17.916505,9.722222,0.100237,11.111111
831,2.943106e+05,0.178015,0.165861,7.908163,14.413265,4.464286,6.586860,4.336735,542.504014,12.755102,1.020408,2.551020,21.101634,25.317188,11.479592,0.110812,9.183673
1983,1.242913e+06,0.142515,0.140300,2.218935,8.284024,6.952663,4.462928,5.325444,1114.859948,21.745562,0.000000,5.917160,60.210265,50.693916,19.230769,0.115194,13.461538


In [643]:
y_train.head()

2266    False
3074     True
2700     True
831      True
1983    False
Name: fake, dtype: bool

## Обучение моделей

In [644]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

In [670]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [671]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### Плохие классфикаторы

- LogisticRegression() 
- SGDClassifier() 
- SVC(gamma='auto') 

### NN - MLPClassifier (f1=0.57) [to be modified]

In [708]:
print_polynomial_pipeline(
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(9, 5, 5, 2), 
        random_state=1, 
        alpha=0.001, 
        max_iter=10000),
    X_train, 
    y_train, 
    X_test, 
    y_test
) 

[[54 51]
 [34 59]]
               precision    recall  f1-score   support

 class "real"       0.61      0.51      0.56       105
class "fakes"       0.54      0.63      0.58        93

     accuracy                           0.57       198
    macro avg       0.57      0.57      0.57       198
 weighted avg       0.58      0.57      0.57       198



### RandomForestClassifier (f1=60..65) [to be modified] 

In [672]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test
) 

[[47 58]
 [28 65]]
               precision    recall  f1-score   support

 class "real"       0.63      0.45      0.52       105
class "fakes"       0.53      0.70      0.60        93

     accuracy                           0.57       198
    macro avg       0.58      0.57      0.56       198
 weighted avg       0.58      0.57      0.56       198



### DecisionTreeClassifier (f1=55..60) [to be modified]

In [680]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=7, min_samples_split=10, min_samples_leaf=15),
    X_train, 
    y_train, 
    X_test, 
    y_test
)

[[51 54]
 [37 56]]
               precision    recall  f1-score   support

 class "real"       0.58      0.49      0.53       105
class "fakes"       0.51      0.60      0.55        93

     accuracy                           0.54       198
    macro avg       0.54      0.54      0.54       198
 weighted avg       0.55      0.54      0.54       198



### RadiusNeighborsClassifier (f1=55..59)

In [704]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=8.9, 
        weights='uniform', 
        p=2, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test
)

[[29 76]
 [ 6 87]]
               precision    recall  f1-score   support

 class "real"       0.83      0.28      0.41       105
class "fakes"       0.53      0.94      0.68        93

     accuracy                           0.59       198
    macro avg       0.68      0.61      0.55       198
 weighted avg       0.69      0.59      0.54       198



### KNeighborsClassifier (f1=0.58) [to be modified]

In [713]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=3, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test
)

[[57 48]
 [33 60]]
               precision    recall  f1-score   support

 class "real"       0.63      0.54      0.58       105
class "fakes"       0.56      0.65      0.60        93

     accuracy                           0.59       198
    macro avg       0.59      0.59      0.59       198
 weighted avg       0.60      0.59      0.59       198

